In [49]:
import math
import pandas as pd 
import numpy as np
from numpy.linalg import inv
from scipy.linalg import eigh
import csv
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [50]:
# % dica - supervised domain-invariant component analysis on colored MNIST
# %
# % Synopsis
# %   [V,D,X,Xt] = dica(Kx, Ky, Kt, groupIdx, lambda, epsilon, M)
# %
# % Description
# %   Domain-invariant component analysis (DICA) finds a low dimensional
# %   subspace of data points from several distributions so as to minimize 
# %   the variance among the distributions of projected data points. It also
# %   takes into account the affinity in the output space.
# % 
# %
# % Inputs ([]s are optional)
# %   (matrix) Kx         NxN kernel matrix between data points
# %   (matrix) Ky         NxN kernel matrix between outputs
# %   (matrix) Kt         NtxN kernel matrix between test samples and
# %                           training samples
# %   (vector) groupIdx   Nx1 vector of group membership of data points
# %   (scalar) lambda     The regularization parameter (input)
# %   (scalar) epsilon    The regularization parameter (output)
# %   (scalar) M          The dimensionality of subspace (M < N)
# %
# % Outputs ([]s are optional)
# %   (matrix) V          Nxdim matrix in which each column is the
# %                       eigenvector
# %   (matrix) D          MxM diagonal matrix in which the diagonal elements
# %                       are eigenvalues associated with the eigenvectors in
# %                       the matrix V
# %   (matrix) X          MxN matrix in which each column is the projection
# %                       of original data point onto the subspace spanned by
# %                       the eigenvectors in the matrix V
# %   (matrix) Xt         MxNt matrix in which each column is the projection
# %                       of test data point onto the subspace spanned by
# %                       the eigenvectors in the matrix V

# % References
# %   K. Muandet, D.Balduzzi,and B.Schölkopf, Domain Generalization via 
# %   Invariant Feature Representation. The 30th International Conference on 
# %   Machine Learning (ICML 2013), pages 10?18, Atlanta, Georgia.
# %
# % DICA Code Reference from
# %   Krikamol Muandet <krikamol@tuebingen.mpg.de>

In [51]:
myfilename = "mnist_digit100_color90flipped_testpurple_022120.npz"
data = np.load(myfilename)

In [52]:
sample = 1000
x_train = data['x_train'][data['train_inds']][:sample]
y_train = data['y_train'][data['train_inds']][:sample]
a_train = data['attr_train'][data['train_inds']][:sample]
x_test = data['x_train'][data['valid_inds']][:sample]
y_test = data['y_train'][data['valid_inds']][:sample]
# x_test = data['x_test'][:sample]
# y_test = data['y_test'][:sample]
x_test.shape

(1000, 2352)

In [53]:
x_train_1_arr = []
y_train_1_arr = []
x_train_2_arr = []
y_train_2_arr = []
for i in range(len(x_train)):
    if np.all(a_train[i] == [1.,0.,0.]): # study 1
        x_train_1_arr.append(x_train[i])
        y_train_1_arr.append(y_train[i])
    elif np.all(a_train[i] == [0.,1.,0.]): # study 2
        x_train_2_arr.append(x_train[i])
        y_train_2_arr.append(y_train[i])
    else:
        raise ValueError()

x_train_1 = np.asarray(x_train_1_arr)
y_train_1 = np.asarray(y_train_1_arr)
x_train_2 = np.asarray(x_train_2_arr)
y_train_2 = np.asarray(y_train_2_arr)
x_train_1.shape

(507, 2352)

In [54]:
def get_variance_x():
    variance_sum = 0
    for j in range(len(x_train[0])):
        variance_sum += np.var(x_train[:,j])
#     for j in range(len(x_train_2[0])):
#         variance_sum += np.var(x_train_2[:,j])

    return 1.0 * variance_sum / (len(x_train[0]))

In [56]:
lmbda = 0.1
eps = 0.001
sigma_x = 5.0 # get_variance_x()
M = 600
sigma_x

5.0

In [57]:
# define kernels

def g_kernel_x(x_p, x_q): # x_p, x_q are images (encoded as 3*28*28 vectors) in i'th domain
    dist = np.linalg.norm(x_p-x_q)**2
#     print(dist)
    power = -1.0/(2.0*(sigma_x**2)) * dist
#     print(power)
    return math.exp(power)

def g_kernel_y(a, b):
    if np.all(a == b):
        return 1
    else:
        return 0

x_p = np.asarray([1]*28 + [1]*(3*28-1)*28)
x_q = np.ones(3*28*28)
print(g_kernel_y(x_p, x_q))

1


In [58]:
# create groupIdx
study_1s = np.asarray([1]*len(x_train_1))
study_2s = np.asarray([2]*len(x_train_2))
groupIdx = np.concatenate((study_1s, study_2s), axis=None)
# groupIdx

In [59]:
n = len(x_train)
s_1 = len(x_train_1)
n

1000

In [60]:
# create k_x

k_x = np.zeros((n, n))
k_x.shape

for i in range(n):
    if i < s_1:
        x_p = x_train_1[i]
    else:
        x_p = x_train_2[i-s_1]
        
    for j in range(n):
        if j < s_1:
            x_q = x_train_1[j]
        else:
            x_q = x_train_2[j-s_1]
        k_x[i][j] = g_kernel_x(x_p, x_q)

In [61]:
# create k_y

k_y = np.zeros((n, n))
k_y.shape

for i in range(n):
    if i < s_1:
        y_p = y_train_1[i]
    else:
        y_p = y_train_2[i-s_1]
        
    for j in range(n):
        if j < s_1:
            y_q = y_train_1[j]
        else:
            y_q = y_train_2[j-s_1]
        k_y[i][j] = g_kernel_y(y_p, y_q)
# k_y[0]

In [62]:
# create k_t

n_t = len(x_test)
k_t = np.zeros((n_t, n))
k_t.shape

for i in range(n_t):
    x_pt = x_test[i]
#     print(x_pt)
        
    for j in range(n):
        if j < s_1:
            x_q = x_train_1[j]
        else:
            x_q = x_train_2[j-s_1]
        k_t[i][j] = g_kernel_x(x_pt, x_q)
# np.all(k_t == np.zeros((n_t, n)))
# print(k_t[:,:10])

In [41]:
# print("training SVM on kernel x...")
# X_train = k_x
# X_test = k_t
# print("scaling data...")
# scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
# X_train = scaling.transform(X_train)
# X_test = scaling.transform(X_test)

# y_train_labels = y_train[:,0]
# y_test_labels = y_test[:,0]
# svclassifier = SVC(kernel='linear')
# svclassifier.fit(X_train, y_train_labels)

# print("evaluating SVM on kernel x...")
# y_pred = svclassifier.predict(X_test)
# print(confusion_matrix(y_test_labels,y_pred))
# print(classification_report(y_test_labels,y_pred))

In [42]:
# print("training SVM on regular X...")
# X_train = x_train
# X_test = x_test
# print("scaling data...")
# scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
# X_train = scaling.transform(X_train)
# X_test = scaling.transform(X_test)

# y_train_labels = y_train[:,0]
# y_test_labels = y_test[:,0]
# svclassifier = SVC(kernel='linear')
# svclassifier.fit(X_train, y_train_labels)

# print("evaluating SVM on regular x...")
# y_pred = svclassifier.predict(X_test)
# print(confusion_matrix(y_test_labels,y_pred))
# print(classification_report(y_test_labels,y_pred))

In [69]:
def dica(Kx, Ky, Kt, groupIdx, lmbda, eps, M):
    N = len(Kx[0])
    Nt = len(Kt[0])
    uniqueGroupIdx = np.unique(groupIdx)
    G = len(uniqueGroupIdx)
    NG = [s_1, n-s_1]


    H = 1.0*np.identity(N)-1.0*np.ones(N)/N

    L = np.zeros((N, N))
    

    for i in range(0, N):
        for j in range(0,N):
            if groupIdx[i] == groupIdx[j]:
                groupSize = NG[groupIdx[i]-1]
                L[i][j] = 1/(G*groupSize*groupSize) - 1/(G*G*groupSize*groupSize)
            else: 
                groupSize_i = NG[groupIdx[i]-1]
                groupSize_j = NG[groupIdx[j]-1]
                L[i][j] = -1.0/(G*G*groupSize_i*groupSize_j)


    Ky = H @ Ky @ H
    Kx = H @ Kx @ H

    B = Ky @ inv(Ky + N*eps*np.identity(N)) @ (Kx @ Kx)
    A = inv(Kx @ L @ Kx + Kx + lmbda*np.identity(N)) @ (B)

    eigenValues, eigenVectors = eigh(A)  # w is the eigenvalues and v are the eigenmatrix, increasing order
    idx = eigenValues.argsort()[::-1] 
    w = eigenValues[idx]
    v = eigenVectors[:,idx]

    print(w)
    V = v[:, :M]
#     print(V)
    D = np.diag(w[:M])
    Evals = np.real(D)

#     print("HULLO " + str((Evals[0][0])))
#     print(V[:,0])
    for i in range(0,M):
        V[:,i] = V[:,i]/(Evals[i,i]**0.5)

#     print(V[:,-1])

    X = V.T @ Kx

    Ht = np.identity(Nt)-np.ones(Nt)/Nt
    Kt = Ht @ Kt @ H
    Xt = V.T @ Kt.T
    return (V, D, X, Xt)

In [70]:
# Nt = 100
# N = 160
# H = 1.0*np.identity(N)-1.0*np.ones(N)/N
# Ht = np.identity(Nt)-np.ones(Nt)/Nt
# Kt = np.zeros((Nt, N))
# Kt.shape
# Kt = np.dot(np.dot(Ht,Kt),H)
# np.dot(Ht,Kt).shape
# H.shape

In [71]:
# k_x.shape

In [72]:
# run DICA to get X and Xt
(V, D, X, Xt) = dica(k_x, k_y, k_t, groupIdx, lmbda, eps, M=400)

[ 7.81678168e+01  2.00230790e+01  1.33240628e+01  9.22748851e+00
  7.14746380e+00  5.89644837e+00  4.71089563e+00  4.28544444e+00
  3.61904975e+00  3.18282231e+00  3.00186478e+00  2.70170108e+00
  2.54598635e+00  2.34802633e+00  2.17134107e+00  2.10443493e+00
  1.91806012e+00  1.76963302e+00  1.67579327e+00  1.66141410e+00
  1.59498283e+00  1.48593981e+00  1.40353964e+00  1.36618651e+00
  1.33854191e+00  1.28711994e+00  1.25195068e+00  1.13156761e+00
  1.09062641e+00  1.06537927e+00  1.02476159e+00  1.00602912e+00
  9.83822311e-01  9.04543784e-01  8.97762112e-01  8.86257658e-01
  8.67486793e-01  8.26286061e-01  8.03604620e-01  7.62169075e-01
  7.42288143e-01  7.35203236e-01  7.12163365e-01  6.71282669e-01
  6.63551636e-01  6.53313816e-01  6.43759973e-01  6.29091061e-01
  6.13622984e-01  5.87890128e-01  5.78953111e-01  5.72343295e-01
  5.59849010e-01  5.52083890e-01  5.51578839e-01  5.49396159e-01
  5.26446886e-01  5.14336356e-01  5.07544123e-01  4.97869540e-01
  4.97069540e-01  4.92260

In [73]:
# input_file = '/Users/rachelh/Programs/rvr/dica_output.npz'
# data = np.load(input_file)
# V = data["V"]
# D = data["D"]
# X = data["X"]
# Xt = data["Xt"]
print(V)
print(X.shape)

[[ 2.85084518e-03 -5.26235457e-03 -5.26907439e-03 ...  1.06623380e-11
   1.48621319e-11  5.95872663e-11]
 [ 9.68003609e-04 -1.75920005e-03 -1.74274654e-03 ... -1.08361915e-10
  -1.51729358e-10 -6.08695800e-10]
 [-1.05258115e-03  1.89176795e-03  1.85958185e-03 ... -5.72273728e-11
  -7.98493444e-11 -3.20562711e-10]
 ...
 [ 9.78868781e-04  2.10853826e-03 -3.16770441e-03 ... -9.81361082e-13
  -3.72793925e-13  2.62203183e-12]
 [-4.23081970e-03 -9.22289981e-03  1.39691950e-02 ... -5.62278844e-11
  -1.69522100e-11 -1.56216399e-11]
 [-1.06087925e-03 -2.31294934e-03  3.50355282e-03 ...  1.15252766e-10
   3.55242752e-11  3.50695440e-11]]
(400, 1000)


In [74]:
# train SVM on X and Xt
small = 1000
X_train = np.transpose(X)[:small]
X_test = np.transpose(Xt)[:small]
print("scaling data...")
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)
print("X_train.shape: " + str(X_train.shape))
# print(X_train)
print("X_test.shape:" + str(X_test.shape))
# print(X_test)

print("training SVM...")
y_train_labels = y_train[:,0]
print("Y_train.shape: " + str(y_train_labels.shape))
# print(y_train_labels)
y_test_labels = y_test[:,0]
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train_labels)

# -----------------------------------------------------
# evaluate SVM
print("evaluating SVM...")
y_pred = svclassifier.predict(X_test)
print(confusion_matrix(y_test_labels,y_pred))
print(classification_report(y_test_labels,y_pred))

scaling data...
X_train.shape: (1000, 400)
X_test.shape:(1000, 400)
training SVM...
Y_train.shape: (1000,)
evaluating SVM...
[[230 273]
 [280 217]]
              precision    recall  f1-score   support

           0       0.45      0.46      0.45       503
           1       0.44      0.44      0.44       497

    accuracy                           0.45      1000
   macro avg       0.45      0.45      0.45      1000
weighted avg       0.45      0.45      0.45      1000



In [255]:
small = 5000
x_train.shape
X = np.zeros((6000, 16000))
np.transpose(X).shape

(16000, 6000)

In [256]:

svclassifier = SVC(kernel='linear')
svclassifier.fit(x_train[:small], y_train_labels[:small])
y_pred = svclassifier.predict(x_test[:small])
print(confusion_matrix(y_test_labels[:small],y_pred))
print(classification_report(y_test_labels[:small],y_pred))

[[325 131]
 [132 412]]
              precision    recall  f1-score   support

           0       0.71      0.71      0.71       456
           1       0.76      0.76      0.76       544

    accuracy                           0.74      1000
   macro avg       0.73      0.74      0.73      1000
weighted avg       0.74      0.74      0.74      1000



In [54]:
# train SVM on X and Xt?
svclassifier = SVC(kernel='linear')
svclassifier.fit(X, y_train_labels)

In [ ]:
# evaluate SVM?

y_pred = svclassifier.predict(Xt)
print(confusion_matrix(y_test_labels,y_pred))
print(classification_report(y_test_labels,y_pred))